# WeatherPy
---------

In [1]:
%load_ext lab_black
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np

# Import citipy to determine city
from citipy import citipy

# import API key
from api_keys import weather_api_key

In [2]:
# Generate cities list
cities = []
lat_lngs = []

# Create set of random lat and lng combos
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for the lat, lng
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # add unique cities
    if city not in cities:
        cities.append(city)

len(cities)

623

In [3]:
# Create lists for the weather findings of each city
city_names = []
country = []
date = []
lat_list = []
lng_list = []
max_temp = []
cloudiness = []
humidity = []
wind_speed = []
index_count = 0
set_count = 1


# Get OpenWeatherMap API
r = requests.get(
    "http://api.openweathermap.org/data/2.5/weather?",
    params={
        "units": "imperial",
        "appid": weather_api_key,
        "q": city,
    },
).json()


print("Beginning Data Retrival")
print("-" * 30)

for index, city in enumerate(cities, start=1):
    try:
        city_names.append(r["name"])
        country.append(r["sys"]["country"])
        date.append(r["dt"])
        lat_list.append(r["coord"]["lat"])
        lng_list.append(r["coord"]["lon"])
        max_temp.append(r["main"]["temp_max"])
        cloudiness.append(r["clouds"]["all"])
        humidity.append(r["main"]["humidity"])
        wind_speed.append(r["wind"]["speed"])
        if index_count > 49:
            index_count = 0
            set_count += 1
        else:
            index_count += 1
        print(f"Processing Record {index_count} of Set {set_count} : {city}")

    except (KeyError, IndexError):
        print("City not found")

print("-" * 30)
print("Data Retrieval Complete")
print("-" * 30)

Beginning Data Retrival
------------------------------
Processing Record 1 of Set 1 : atuona
Processing Record 2 of Set 1 : lebu
Processing Record 3 of Set 1 : khatanga
Processing Record 4 of Set 1 : mendeleyevsk
Processing Record 5 of Set 1 : albany
Processing Record 6 of Set 1 : borovoy
Processing Record 7 of Set 1 : busselton
Processing Record 8 of Set 1 : kutum
Processing Record 9 of Set 1 : torbay
Processing Record 10 of Set 1 : taolanaro
Processing Record 11 of Set 1 : nabire
Processing Record 12 of Set 1 : port elizabeth
Processing Record 13 of Set 1 : thompson
Processing Record 14 of Set 1 : saint-philippe
Processing Record 15 of Set 1 : tasiilaq
Processing Record 16 of Set 1 : mizpe ramon
Processing Record 17 of Set 1 : dakar
Processing Record 18 of Set 1 : kapaa
Processing Record 19 of Set 1 : yumen
Processing Record 20 of Set 1 : mingshui
Processing Record 21 of Set 1 : gizo
Processing Record 22 of Set 1 : wanxian
Processing Record 23 of Set 1 : cidreira
Processing Record 24